In [1]:
#imports
import os
import yaml
import json

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
#params
dt_cols = ['t_start', 't_end']
price_cols = ['open_price','close_price','low_price','high_price']
feature_cols = ['base_volume','buy_base','n_trades']

shift_b = 18
shift_f = 7

anomaly_crtiretion = 1.0050

In [3]:
#read
df = pd.read_excel('dataset_before_features.xlsx', index_col =0)

In [4]:
#deltas
for col in feature_cols:
    df[f'{col}_delta'] = df[col].shift(-1) - df[col]
condition = (df['is_closed'] == True)
delta_cols = [col + '_delta' for col in feature_cols]
for col_1, col_2 in zip(feature_cols,delta_cols):
    df[col_2].loc[condition] = (df[col_1].loc[condition].values) + (df[col_2].loc[condition].values)
df = df.drop(axis=0, index = [1774,79063])
df = df.drop('is_closed', axis = 1)

#vol_per_trade
df['vol_per_trade_delta'] = df['base_volume_delta'] / df['n_trades_delta']
delta_cols.append('vol_per_trade_delta')

#shift deltas
data_temp = pd.DataFrame()
for col in delta_cols:
    for shift in range(1,shift_b+1):
        data_temp[f'{col}_{shift}'] = df[col].shift(shift)
df = pd.concat([df, data_temp], axis = 1)

#shift cumsum deltas
data_temp = pd.DataFrame()
for col in delta_cols:
    for shift in range(2,shift_b+1):
        data_temp[f'{col}_0_{shift-1}'] = df[col].rolling(shift).sum()
df = pd.concat([df, data_temp], axis = 1)

#target
df['anomaly_t_start'] = np.where(
    df.close_price > df.open_price,
    df.high_price/df.open_price,
    df.open_price/df.low_price
    )
df['anomaly_t_end'] = df['anomaly_t_start'].shift(-shift_f)
df['target'] = df['anomaly_t_end'] > anomaly_crtiretion

In [5]:
###prepare
df = df.dropna()

###select
delta_cols = df.drop(dt_cols+price_cols+feature_cols, axis = 1).columns.to_list()
df_event = df[dt_cols + price_cols + ['target']].reset_index(drop = True)
df_model = df[delta_cols + ['target']].reset_index(drop = True)